In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
print('Welcome to the OK_Cupid data scientist project!\n')

`Reading the data from the csv file and making a pandas dataframe of it`

In [ ]:
from my_python.filefinder import helper

In [ ]:
df = helper.stat_display()

`Gathering some info about the dataframe`

In [ ]:
'Unique values of income: ', df.income.unique()

In [ ]:
df[df.income == -1].head(1)

In [ ]:
print('The number of observations with income value -1 is:', len(df[df.income == -1]))

### We could drop the whole income column because -1 is not a usual categorical variable and definitely not a continuous numerical variable. Also, we can replace the values without avoiding bias.

In [ ]:
df = df.drop('income', axis=1)

In [ ]:
#df.religion.unique()
df.pets.unique()

In [ ]:
'Unique values of height: ', df.height.unique()

In [ ]:
print('''\nThe smallest 18 year-old-man in the world is 29 inches, thus we can replace, or drop the values if they are lower than that.
https://www.mirror.co.uk/news/world-news/worlds-shortest-teenager-named-18-27050545''')

In [ ]:
invalid_heights = (df.height <= 29) | (df.height.isna())

In [ ]:
df = df[~invalid_heights]

In [ ]:
df.describe()

In [ ]:
df.height.unique()

`Converting inch to cm`

In [ ]:
df.height = df.height*2.54

`Take a look at the "nan" values of the whole dataframe`

In [ ]:
df_isna = df.isna().sum().sort_values()
#df_isna

`Choosing some categorical and numerical variables (with zero or little null values) and put them into a list`

In [ ]:
cats_and_nums = ['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'height', 'orientation', 'religion', 'sex', 'sign', 'smokes', 'status']

In [ ]:
df.sex.replace(['m', 'f'], ['male', 'female'], inplace=True)

In [ ]:
#for feature in cats_and_nums:
    #print(df[feature].value_counts())

In [ ]:
#df = df[cats_and_nums].dropna()

In [ ]:
from my_python.methods import df_cleaning as to_clean

In [ ]:
changes = to_clean.unique_and_fill(cats_and_nums, df)

In [ ]:
while True:
    try: x = next(changes)
    except StopIteration: break
    #print(x)

`Making a function which deals with features having many unique values in a way of making groups.`

`Making groups from signs, diets and religions by using the above function`

In [ ]:
signs_ok = ['gemini', 'cancer', 'pisces', 'aquarius', 'taurus', 'virgo', 'sagittarius', 'leo', 'aries', 'libra', 'scorpio', 'capricorn']
sings_change = to_clean.change_values(signs_ok, df, 'sign')
diets_ok = ['kosher', 'halal', 'anything', 'other', 'vegetarian', 'vegan']
diets_change = to_clean.change_values(diets_ok, df, 'diet')
religions_ok = ['agnosticism', 'atheism', 'christianity', 'other', 'catholicism', 'buddhism', 'judaism', 'hinduism', 'islam']
religion_change = to_clean.change_values(religions_ok, df, 'religion')
sings_change

`Manual data manipulation of education unique values`

In [ ]:
edu_unique = ['working on college/university', 'working on space camp', 'graduated from masters program', 'graduated from college/university',
'working on two-year college', 'unknown', 'graduated from high school', 'working on masters program', 'graduated from space camp',
'college/university', 'dropped out of space camp', 'graduated from ph.d program', 'graduated from law school', 'working on ph.d program',
'two-year college','graduated from two-year college', 'working on med school', 'dropped out of college/university', 'space camp',
'graduated from med school', 'dropped out of high school', 'working on high school', 'masters program', 'dropped out of ph.d program',
'dropped out of two-year college', 'dropped out of med school', 'high school', 'working on law school', 'law school',
'dropped out of masters program', 'ph.d program', 'dropped out of law school', 'med school']

In [ ]:
high_school = ['working on college/university', 'working on two-year college', 'graduated from high school', 'working on masters program',
'high school','working on law school', 'working on high school']
college = ['graduated from law school', 'two-year college', 'graduated from two-year college','working on med school', 'law school']
master = ['graduated from masters program', 'graduated from college/university', 'college/university', 'working on ph.d program',
'masters program']
phd = ['graduated from ph.d program', 'graduated from med school', 'ph.d program', 'med school']
other = ['working on space camp', 'graduated from space camp', 'space camp']
dropped = ['dropped out of space camp', 'dropped out of college/university', 'dropped out of high school', 'dropped out of ph.d program',
'dropped out of two-year college', 'dropped out of med school', 'dropped out of masters program','dropped out of law school']

In [ ]:
all_in = [high_school, college, master, phd, other, dropped]
new = ['high school', 'college', 'master', 'phd', 'other', 'dropped out']
edu_test = to_clean.list_replacement(df, 'education', all_in, new)

`Making pie charts, countplots and histograms of the features from the list`

In [ ]:
from my_python.methods import visual as vis

In [ ]:
print('\n')
question = input('Would you like to see some pie charts and countplots? Enter "yes", or "no"! ')
if question == 'yes':
    vis.pie_and_countplot(cats_and_nums, df, 'sex')

In [ ]:
print('\n')
question = input('Would you like to see some histograms? Enter "yes", or "no"! ')
if question == 'yes':
    vis.histogram(cats_and_nums, df, 'sex')

`Making some dataframes by copying the original`

In [ ]:
df_model = df.copy()
df_new = df.copy()

In [ ]:
df_model.head(1)

In [ ]:
df_model.sex = df_model.sex.map({'male':1, 'female':0})
df_model.orientation = df_model.orientation.replace(['straight', 'bisexual', 'gay'], [2, 1, 0])
df_model.status = df_model['status'].replace(['single', 'available'], 1)
df_model.status = df_model['status'].replace(['married', 'seeing someone', 'unknown'], 0)

df_model.smokes = np.where(df_model.smokes == 'no', 1, 0)
df_model.drugs = np.where(df_model.drugs == 'never', 1, 0)

`Making some labels automatically`

In [ ]:
encodeable = [feat for feat in cats_and_nums if feat not in ['sex', 'orientation', 'status', 'smokes', 'drugs', 'age', 'height']]

In [ ]:
for uncoded in encodeable:
    encoder = LabelEncoder()
    df_model[uncoded] = encoder.fit_transform(df_model[uncoded])

In [ ]:
df_model = df_model[cats_and_nums]

In [ ]:
#scaler = StandardScaler()

In [ ]:
#df_model = scaler.fit_transform(df_model)
#df_model

`Making a function to build and use LogisticRegression model.`

In [ ]:
import pandas as pd

In [ ]:
def model_log(df: pd.DataFrame, target: str) -> dict:
    
    df = df[df[target] != 'unknown']
    df = df.sample(int(.25*len(df)))
    X = df.drop(target, axis=1)
    y = df[target]
    scaler = StandardScaler()
    scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=15)
    model = LogisticRegression(max_iter=3500, solver='saga')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return {'Logistic model': model._estimator_type, 'Features': list(X.columns), 'Target': target, 'Accuracy:': '%.3f' % accuracy}

In [ ]:
log_test_sign = model_log(df_model, 'sign')

In [ ]:
print(log_test_sign)

In [ ]:
##encoder = LabelEncoder()
#df.sign = encoder.fit_transform(df.sign)

In [ ]:
target = 'orientation'
sample = df_model.sample(10000, replace=True)
X = sample.drop(target, axis=1)
y = sample[target]

In [ ]:
X

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [ ]:
bag_dt = BaggingClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=10)
bag_dt.fit(x_train, y_train)
bag_accuracy = bag_dt.score(x_test, y_test)
print('Accuracy score of Bagged Classifier, 10 estimators: %.3f' % bag_accuracy)

In [ ]:
np.random.seed(42)
accus_test = []
neighbors = range(1,50)
for neighbor in neighbors:
    model = KNeighborsClassifier(n_neighbors=neighbor)
    StandardScaler().fit_transform(X)
    model.fit(x_train, y_train)
    y_pred_test = model.predict(x_test)
    accus_test.append(accuracy_score(y_test, y_pred_test))
best_acc_neighbors = np.max(accus_test)
best_neighbors = neighbors[np.argmax(accus_test)]
print('Best accuracy with using k Nearest neighbors is: %.3f' % best_acc_neighbors, ' and the best number of neighbors is: ', best_neighbors)

In [ ]:
np.random.seed(42)
accuracy_train=[]
accuracy_test = []
depths = range(1,20)
for depth in depths:
  model = RandomForestClassifier(max_depth=depth)
  model.fit(x_train, y_train)
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)
  accuracy_train.append(accuracy_score(y_train, y_pred_train))
  accuracy_test.append(accuracy_score(y_test, y_pred_test))
best_acc = np.max(accuracy_test)
best_depth = depths[np.argmax(accuracy_test)]
print('Best accuracy with using Randomforest is: ', best_acc, ' and the best depth is: ', best_depth)

In [ ]:
vis.randfor_test_plot(depths, accuracy_train, accuracy_test)

In [ ]:
best_rf = RandomForestClassifier(max_depth=best_depth)
best_rf.fit(x_train, y_train)

In [ ]:
feature_importance = pd.DataFrame(zip(x_train.columns, best_rf.feature_importances_), columns=['features', 'importances']).sort_values('importances', ascending=False)
feature_importance.iloc[:5]

In [ ]:
features = ['sign', 'age', 'height', 'sex', 'orientation', 'body_type', 'diet', 'drinks',  'smokes', 'drugs', 'education', 'religion', 'status']

In [ ]:
df_new = df_new[features][df_new.sign != 'unknown']

In [ ]:
X_new = pd.get_dummies(df_new[features[1:]], drop_first=True)
X_new

In [ ]:
y_new = df_new[features[0]]

In [ ]:
y_new

In [ ]:
neighbors = KNeighborsClassifier(n_neighbors=1)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_new = scaler.fit_transform(X_new)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_new, y_new, test_size=.25, random_state=100)

In [ ]:
neighbors.fit(x_train, y_train)

In [ ]:
y_pred = neighbors.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
np.random.seed(0)
accus_test = []
neighbors = range(1,50)
for neighbor in neighbors:
    model = KNeighborsClassifier(n_neighbors=neighbor)
    model.fit(x_train, y_train)
    y_pred_test = model.predict(x_test)
    accus_test.append(accuracy_score(y_test, y_pred_test))
best_acc_neighbors = np.max(accus_test)
best_neighbors = neighbors[np.argmax(accus_test)]
print('Best accuracy with using k Nearest neighbors is: ', best_acc_neighbors, ' and the best number of neighbors is: ', best_neighbors)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
array = np.array(X_new)
array